In [1]:
import glob
import os
import warnings

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from shapely.geometry import Point

warnings.filterwarnings("ignore")

**Extract PSC within segments**

Input file:
- segment shapefiles
    - segment file required column: C_Length (cumulative length) & Name (Seg_{!C_Length!})
- ps point data shapefile

Implemented fuction:
- `PSC_Within_Profile_Segment()`

In [2]:
def export_parameter_record(savefolder, params_dict):

    import os

    for i in range(1, 1000):
        savepath = os.path.join(savefolder, "Param_Record_{}.txt".format(str(i).zfill(3)))
        if os.path.exists(savepath):
            continue
        else:
            break

    with open(savepath, "w+") as file:
        for key, value in params_dict.items():
            file.write("{} : {}".format(key, value))
            file.write("\n\n")
    pop_up_string = "Saved at {}".format(savefolder)
    print(pop_up_string)


def open_geodata(filepath):
    import sys

    import geopandas as gpd

    geodata_type = type(gpd.GeoDataFrame())

    if type(filepath) == geodata_type:
        geodata = filepath
    elif type(filepath) == str:
        extension = filepath.split(".")[-1]
        if extension == "pkl":
            geodata = pd.read_pickle(filepath, compression="zip")
        elif extension == "shp":
            geodata = gpd.read_file(filepath)
        else:
            sys.exit(
                "The datatype of PSC_filepath is invalid.\
            \nExpect {} or {}, not {}".format(
                    type("1"), geodata_type, type(PSC_filepath)
                )
            )
    else:
        sys.exit("{} invalid filetype".format(type(filepath)))
    return geodata


def PSC_Within_Profile_Segment(psc_filepath, segment_filepath, distance=200):
    # ---------------------------------------------
    import warnings

    warnings.filterwarnings("ignore")

    import os
    import sys

    import geopandas as gpd
    import pandas as pd
    from tqdm import tqdm, trange

    # ---------------------------------------------
    radius = distance

    # đọc shapefile của PS candidates

    PSC_data = open_geodata(psc_filepath)

    if PSC_data.crs == None:
        PSC_data.crs = "epsg:3826"

    # đọc shapefile của profile segment
    segment_data = open_geodata(segment_filepath)
    segment_data_length = len(segment_data)

    # một dataframe trống để chứa thông tin các điểm bên trong buffer
    empty_table = pd.DataFrame(data=None)

    for i, _temp in zip(range(segment_data_length), trange(segment_data_length)):
        try:
            # profile_segment = segment_data.loc[i, :]
            # profile_segment_buffer = profile_segment.geometry.buffer(radius)

            profile_segment_buffer = segment_data.loc[i, :].geometry

            psc_within_buffer = PSC_data.geometry.within(profile_segment_buffer)
            selected_PSC_table = PSC_data[psc_within_buffer]

            segment_name = segment_data.loc[i, "Name"]
            selected_PSC_table["Segment"] = segment_name

            empty_table = pd.concat([empty_table, selected_PSC_table])
        except Exception as e:
            sys.exit("{} at location {}".format(e, i))

    empty_table.reset_index(inplace=True, drop=True)

    new_geospatial_datatable = gpd.GeoDataFrame(
        data=empty_table, geometry=empty_table.geometry, crs=empty_table.crs
    )

    return new_geospatial_datatable

#### Input Data

In [3]:
geodata_filepath = r"D:\SENTINEL_1A_SAR_IMAGES\030_REPROCESS_CHOUSHUI_P2\CALIBRATION_PROCESS\Calibrated_Files\LOS_mm_ALL_POINTS_v010\VERT_mm\VERT_CALIBRATED_CUMDISP_mm_HSR_BUFFER\VERT_CALIBRATED_CUMDISP_mm_HSR_BUFFER.pkl"
segment_shp_filepath = r"E:\002_ARCGIS_WORK\ManuscriptArcGIS\HSR_Shapefiles\BUFFER\SELECT_BUFFER\BUFFER_CrossingLine_Segmented_W50_L5000.shp"

savefolder = r"D:\SENTINEL_1A_SAR_IMAGES\030_REPROCESS_CHOUSHUI_P2\CALIBRATION_PROCESS\Calibrated_Files\LOS_mm_ALL_POINTS_v010\VERT_mm\HSR_PROFILE\CROSSTrackProfile"
extension_option = "pkl"

if not os.path.exists(savefolder):
    os.makedirs(savefolder)

params_dict = {
    "geodata_filepath": geodata_filepath,
    "segment_shp_filepath": segment_shp_filepath,
    "savefolder": savefolder,
}

#### Data Processing

In [4]:
segment_shp_data = gpd.read_file(segment_shp_filepath)

unique_linename = segment_shp_data["LineName"].unique()

cache = []

for line_name in unique_linename:

    select_segment_data = segment_shp_data[segment_shp_data["LineName"] == line_name]
    select_segment_data = select_segment_data.reset_index(drop=True)

    psc_data = PSC_Within_Profile_Segment(
        psc_filepath=geodata_filepath, segment_filepath=select_segment_data
    )

    psc_data["LineName"] = line_name

    columns = psc_data.columns.tolist()
    new_columns = columns[:-3] + columns[-3:][::-1]

    output_table = psc_data[new_columns]

    cache.append(output_table)

 99%|████████████████████████████████████████████████████████████████████████████████▏| 99/100 [00:05<00:00, 17.90it/s]


#### Save Data

In [5]:
output_data = pd.concat(cache, ignore_index=True)
base = os.path.basename(segment_shp_filepath).split(".")[0]

if extension_option == "pkl":
    savename = "Output_" + base + ".pkl"
    savepath = os.path.join(savefolder, savename)
    output_data.to_pickle(savepath, compression="zip")
elif extension_option == "shp":
    savename = "Output_" + base + ".shp"
    savepath = os.path.join(savefolder, savename)
    output_data.to_file(savepath)
else:
    raise ("Invalid filetype!")

export_parameter_record(savefolder=savefolder, params_dict=params_dict)

Saved at D:\SENTINEL_1A_SAR_IMAGES\030_REPROCESS_CHOUSHUI_P2\CALIBRATION_PROCESS\Calibrated_Files\LOS_mm_ALL_POINTS_v010\VERT_mm\HSR_PROFILE\CROSSTrackProfile
